<a href="https://colab.research.google.com/github/Yue1Harriet1Huang/news-dashboard-app/blob/master/connect_to_azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-common
!pip install azure-mgmt-sql
!pip install azure-mgmt-resource

In [2]:
from azure.mgmt.resource import ResourceManagementClient
'fl0qDqm6Y0t87zt6W+n2rb0actEFhyrs6MqEZn8ZVNPhaY+WVLtm/8jjWaoCWTMvPKQ+ATbqaVjQFkwduXTp3Q=='
AccountName = 'newsdashboard'
AccountKey='fl0qDqm6Y0t87zt6W+n2rb0actEFhyrs6MqEZn8ZVNPhaY+WVLtm/8jjWaoCWTMvPKQ+ATbqaVjQFkwduXTp3Q=='
EndpointSuffix='core.windows.net'

RESOURCE_GROUP = 'datastorage'
LOCATION = 'eastCanada East, Canada Central'  # example Azure availability zone, should match resource group
#SQL_SERVER = 'yourvirtualsqlserver'
#SQL_DB = 'YOUR_SQLDB_NAME'
#USERNAME = 'YOUR_USERNAME'
#PASSWORD = 'YOUR_PASSWORD'




In [3]:
!pip install azure-storage-file-datalake

In [4]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings

In [5]:
#Connect by using Azure Active Directory (Azure AD)

AccountName = 'newsdashboard'
AccountKey='fl0qDqm6Y0t87zt6W+n2rb0actEFhyrs6MqEZn8ZVNPhaY+WVLtm/8jjWaoCWTMvPKQ+ATbqaVjQFkwduXTp3Q=='
EndpointSuffix='core.windows.net'


def initialize_storage_account(storage_account_name, storage_account_key):
    
    try:  
        global service_client

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    
    except Exception as e:
        print(e)

def create_file_system():
    try:
        global file_system_client

        file_system_client = service_client.create_file_system(file_system="my-file-system")
    
    except Exception as e:
        print(e)

def create_directory():
    try:
        file_system_client.create_directory("my-directory")
    
    except Exception as e:
     print(e)

def upload_file_to_directory():
    try:

        file_system_client = service_client.get_file_system_client(file_system="my-file-system")

        directory_client = file_system_client.get_directory_client("my-directory")
        
        file_client = directory_client.create_file("uploaded-file.txt")
        local_file = open("C:\\file-to-upload.txt",'r')

        file_contents = local_file.read()

        file_client.append_data(data=file_contents, offset=0, length=len(file_contents))

        file_client.flush_data(len(file_contents))

    except Exception as e:
      print(e)

def upload_file_to_directory_bulk():
    try:

        file_system_client = service_client.get_file_system_client(file_system="my-file-system")

        directory_client = file_system_client.get_directory_client("my-directory")
        
        file_client = directory_client.get_file_client("uploaded-file.txt")

        local_file = open("C:\\file-to-upload.txt",'r')

        file_contents = local_file.read()

        file_client.upload_data(file_contents, overwrite=True)

    except Exception as e:
      print(e)
      

In [6]:
initialize_storage_account(AccountName, AccountKey)


In [14]:
def create_directory(file_system_name, directory_name):
    file_system_client = service_client.get_file_system_client(file_system=file_system_name)

    try:
        file_system_client.create_directory(directory_name)
    
    except Exception as e:
     print(e)

create_directory(file_system_name="news", directory_name="data")

In [9]:
def initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
    except Exception as e:
        print(e)

#initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id)

In [11]:
def list_directory_contents(file_system_name, directory):
    try:
        
        file_system_client = service_client.get_file_system_client(file_system=file_system_name)

        paths = file_system_client.get_paths(path=directory)

        for path in paths:
            print(path.name + '\n')

    except Exception as e:
     print(e)

list_directory_contents(file_system_name="news", )

(FilesystemNotFound) The specified filesystem does not exist.
RequestId:5b814708-301f-000e-36dc-21b1ac000000
Time:2021-03-26T01:07:20.6773654Z
